In [ ]:
import numpy as np
import matplotlib.pyplot as plt





In [ ]:
dataset = np.load("dataset.npz")
X, y = dataset["X"], dataset["y"]

In [ ]:
#vislualize

#chosen image
k=10

shape = (X[k].reshape(20,20))
flatten = shape.flatten()


plt.imshow(shape, vmin=0, vmax=255, cmap="gray")
plt.show()

In [ ]:
dataset = np.load("dataset_corrupted.npz")
X, y = dataset["X"], dataset["y"]